# Lung Cancer Classification using Computerized Tomography
 Project developed by: **Eduardo Passos** [202205630](sigarra.up.pt/fcup/fest_geral.cursos_list?pv_num_unico=202205630), **Pedro Fernandes** [202208347](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202208347) and **Rafael Pacheco** [202206258](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202206258)


### Index {#index} #############################################
1. [Project Introduction](#intro)




## 1.Project Introduction and Motivation {#intro}
[Back to Index](#index)

This project focuses on classifying lung cancer using Computed Tomography (CT) scan data, using the [LIDC-IDRI](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966254) dataset. 

As referenced in the assignment, **lung cancer leads global cancer-related mortality rates**, with only 16% of cases detected at a localized stage. In these instances, patients have a five-year survival rate exceeding 50%. However, when diagnosed at an advanced stage, the five-year survival rate drops to just 5%. Therefore, **early diagnosis is crucial for improving survival outcomes**, and systems designed to assist with screening can be instrumental regarding this matter (statistics from [World Health Organization](https://www.iarc.who.int/featured-news/latest-global-cancer-data-cancer-burden-rises-to-18-1-million-new-cases-and-9-6-million-cancer-deaths-in-2018/) and [American Cancer Society](https://www.cancer.org/research/cancer-facts-statistics/all-cancer-facts-figures/cancer-facts-figures-2019.html)).

The goal is to **develop a machine learning model capable of accurately identifying lung cancer** (based on the provided medical imaging data) in early stages of the tumour, in order to prevent evolution of the disease. To support this, we have developed a program that **converts the dataset** (initially in DICOM format) into numpy array files (.npy), while extracting and storing key information in a CSV file. The preprocessed data will be utilized to **train the ??????????????????????????**, enabling accurate classification of the nodules from the imaging data.

Lung cancer leads global cancer-related mortality rates [Wor18, Ame19], with only 16% of cases detected at a localized stage. In these instances, patients have a five-year survival rate exceeding 50%. However, when diagnosed at an advanced stage, the five-year survival rate drops to just 5%. Therefore, early diagnosis is crucial for improving survival outcomes, and systems designed to assist with screening could be instrumental in this effort.